Reads logs created by the secondary task app (containing the CMU app logs + sec task logs), extracts primary task info and secondary task info, and puts the data in the Postgresql database. 

NOTE: before running this script you should ensure that the database existis and that it contains tables primary_task and secondary_task. File table_create.sql contains commands for creating the necessary tables. File tables_alter.sql adds the missing fields to primary_task.

SEED_TASK_ID and SEED_SEC_TASK_ID should be set to the last prim/sec task ID added to the database. 

NOTE: check the SQL statements. Either those for inserting or those for updating the data should be uncommented. 

NOTE: check the pointer to the directory where the raw logs are stored 

In [31]:
import psycopg2
import sys
import re
import os
from datetime import datetime

In [32]:
# Reset the whole process - taskIds start from zero
def reset_global():
    global SEED_TASK_ID
    global SEED_SEC_TASK_ID
    SEED_TASK_ID = 0 # 224 - this is where IJS data should start from
    SEED_SEC_TASK_ID = 0 # 1730 - this is where IJS data should start from

def reset_user():
    global client_id
    client_id = ""

# For each new Task - reset all task-related params
def reset_task():
    global start_time
    global end_time
    global state_prim
    global state_sec
    global task_type
    global label
    global finished
    global num_all_correct
    global num_correct
    global num_incorrect
    global curr_task_id
    global curr_sec_task_id
    global task_load_index
    global old_task_load_index
    global mental
    global physical
    global temporal
    global performance
    global effort
    global frustration

    start_time = 0
    end_time = 0
    num_correct = 0
    num_incorrect = 0
    num_all_correct = 0
    label = ""
    task_type = ""
    task_load_index = 0
    old_task_load_index = 0
    mental = 0
    physical = 0
    temporal = 0
    performance = 0
    effort = 0
    frustration = 0    
    finished = True
    state_prim = 0
    state_sec = 0
    curr_task_id = -1
    curr_sec_task_id = -1


In [40]:
# CHANGE YOUR DB PARAMETERS HERE
dsn_database = "veljkodb"
dsn_hostname = "127.0.0.1"
dsn_port = "5432"
dsn_uid = ""     
dsn_pwd = ""

In [34]:
def connect_to_db():
    global cursor
    global conn
    try:
        conn_string = "host="+dsn_hostname+" port="+dsn_port+" dbname="+dsn_database+" user="+dsn_uid+" password="+dsn_pwd
        print "Connecting to database\n  ->%s" % (conn_string)
        conn=psycopg2.connect(conn_string)
        cursor = conn.cursor()
        print "Connected!\n"
    except:
        print "Unable to connect to the database."
    return cursor

In [35]:
# Each line of text in the file should be processed, so that primary and secondary tasks are identified

def parse(line):
    global curr_task_id
    global curr_sec_task_id
    global num_correct 
    global num_incorrect
    global label
    global finished
    global task_load_index
    global old_task_load_index
    global mental
    global physical
    global temporal
    global performance
    global effort
    global frustration    
    global start_time
    global start_time_sec
    global end_time
    global state_prim
    global state_sec
    global task_type
    global cursor
     
    mHP = re.compile(r"(\S+), Hidden Pattern Question Slide, HiddenPattern(\d).txt, (\w+)," )
    mHPres = mHP.search(line)
    if mHPres and state_prim == 0:
        task_type = 'HP'
        label = mHPres.group(3).lower()
        start_time = convert_to_epoch(mHPres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id 
 
    mFA = re.compile(r"(\S+), Finding A's Question Slide, FindingAs_(\d).txt, (\w+)" )
    mFAres = mFA.search(line)
    if mFAres and state_prim == 0:
        task_type = 'FA'
        label = mFAres.group(3).lower()
        start_time = convert_to_epoch(mFAres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id 
        
    mGC = re.compile(r"(\S+), Gestalt Completion Question Slide, GestaltCompletion(\d).txt, (\w+)" )
    mGCres = mGC.search(line)
    if mGCres and state_prim == 0:
        task_type = 'GC'
        label = mGCres.group(3).lower()
        start_time = convert_to_epoch(mGCres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id         
    
    mNC = re.compile(r"(\S+), Number Comparison Question Slide, NumberComparison_(\d).txt, (\w+)" )
    mNCres = mNC.search(line)
    if mNCres and state_prim == 0:
        task_type = 'NC'
        label = mNCres.group(3).lower()
        start_time = convert_to_epoch(mNCres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id     
    
    mSX = re.compile(r"(\S+), Scattered X's Question Slide, ScatteredXs_(\d).txt, (\w+)" )
    mSXres = mSX.search(line)
    if mSXres and state_prim == 0:
        task_type = 'SX'
        label = mSXres.group(3).lower()
        start_time = convert_to_epoch(mSXres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id     

    mPT = re.compile(r"(\S+), Pursuit Test Question Slide, Pursuit_(\d).txt, (\w+)" )
    mPTres = mPT.search(line)
    if mPTres and state_prim == 0:
        task_type = 'PT'
        label = mPTres.group(3).lower()
        start_time = convert_to_epoch(mPTres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id        
    
    mHProw = re.compile(r"'image/HiddenPatterns/(\S+)_(\w+).jpg' selected")
    mHProwres = mHProw.search(line)
    if mHProwres:
        result = mHProwres.group(2)
        if result == "true":
            num_correct += 1
        elif result == "false":
            num_incorrect += 1

    mFArow = re.compile(r" '([a-zA-Z]+)' selected")
    mFArowres = mFArow.search(line)
    if mFArowres:
        result = mFArowres.group(1).lower()
        if 'a' in result:
            num_correct += 1
        else:
            num_incorrect += 1
    
    mNCrow = re.compile(r" '([0-9]+)-([0-9]+)' selected")
    mNCrowres = mNCrow.search(line)
    if mNCrowres:
        num1 = mNCrowres.group(1)
        num2 = mNCrowres.group(2)
        if num1 != num2:
            num_correct += 1
        else:
            num_incorrect += 1
        
    mSXrow = re.compile(" \(\d+,\d+\), \(\d+,\d+\)")
    mSXrowres = mSXrow.search(line)
    if mSXrowres:
        num_correct += 1
        
    mPTrow = re.compile(", (\d+), '(\d+)'")
    mPTrowres = mPTrow.search(line)
    if mPTrowres:
        num1 = mPTrowres.group(1)
        num2 = mPTrowres.group(2)
        if PT_checker(num1, num2, label):
            num_correct += 1
        else:
            num_incorrect += 1

    mTIU = re.compile(r"TimeIsUp Slide")
    mTIUres = mTIU.search(line)
    if mTIUres:
        finished = False 
        
    mR = re.compile(r"(\S+), Rating Slide")
    mRres = mR.search(line)
    if mRres:
        end_time = convert_to_epoch(mRres.group(1))
        
    mRM = re.compile(r"(Mental|Physical|Temporal|Performance|Effort|Frustration), ([\s\w]+)")
    mRMres = mRM.search(line)
    if mRMres:
        if mRMres.group(1).rstrip() == "Mental":
            mental = TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())
        elif mRMres.group(1).rstrip() == "Physical":
            physical = TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())
        elif mRMres.group(1).rstrip() == "Temporal":
            temporal = TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())
        elif mRMres.group(1).rstrip() == "Performance":
            performance = TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())
        elif mRMres.group(1).rstrip() == "Effort":
            effort = TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())
        elif mRMres.group(1).rstrip() == "Frustration":
            frustration = TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())
        task_load_index = mental + physical + temporal + performance + effort + frustration    
        old_task_load_index += TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())

    
    mB = re.compile(r"(Break|End) Slide")
    mBres = mB.search(line)
    if mBres and state_prim == 1:
        process_end()
        reset_task()
        
    mSCW = re.compile(r"(\d+):showing ColorWindow")
    mSCWres = mSCW.search(line)
    if mSCWres:
        if state_sec == 1:
            if curr_task_id > -1:
                print "Secondary task", curr_sec_task_id, curr_task_id, client_id, start_time_sec, "False"            
                #if cursor != None:
                #    cursor.execute("""INSERT INTO secondary_task (sec_task_id, prim_task_id, client_id, timestamp, opacity, finished) VALUES (%s, %s, %s, %s, %s, %s);""", (curr_sec_task_id,curr_task_id,client_id,start_time_sec,0.0,"false"))
        curr_sec_task_id = generate_sec_task_ID()
        start_time_sec = int(mSCWres.group(1))*1000
        state_sec = 1
    
    mSUC = re.compile("(\d+):user clicked on rectangle. opacity level: (\d+\.\d+)")
    mSUCres = mSUC.search(line)
    if mSUCres:
        if state_sec == 1 and curr_task_id > -1:
            opacity = mSUCres.group(2)
            state_sec = 0
            print "Secondary task", curr_sec_task_id, curr_task_id, client_id, start_time_sec, opacity, "True"
            #if cursor != None:
            #    cursor.execute("""INSERT INTO secondary_task (sec_task_id, prim_task_id, client_id, timestamp, opacity, finished) VALUES (%s, %s, %s, %s, %s, %s);""", 
            #        (curr_sec_task_id,
            #        curr_task_id,
            #        client_id,
            #        start_time_sec,
            #        opacity,
            #        "true"))            


In [36]:
def generate_task_ID():
    global SEED_TASK_ID
    SEED_TASK_ID += 1
    return SEED_TASK_ID

def generate_sec_task_ID():
    global SEED_SEC_TASK_ID
    SEED_SEC_TASK_ID += 1
    return SEED_SEC_TASK_ID

# Result checker for PT task
def PT_checker(num1, num2, task_type):

    low_map = {
        "1": "3",
        "2": "4",
        "3": "1",
        "4": "2",
        "5": "6",
        "6": "5",
        "7": "9",
        "8": "10",
        "9": "7",
        "10": "8",
    }

    medium_map = {
        "1": "5",
        "2": "6",
        "3": "2",
        "4": "8",
        "5": "10",
        "6": "1",
        "7": "3",
        "8": "9",
        "9": "4",
        "10": "6",
    }    
    
    high_map = {
        "1": "9",
        "2": "8",
        "3": "10",
        "4": "7",
        "5": "6",
        "6": "5",
        "7": "4",
        "8": "3",
        "9": "1",
        "10": "2",
    }    
    
    if task_type == "low":
        return num2 == low_map.get(num1)
    elif task_type == "medium":
        return num2 == medium_map.get(num1)
    elif task_type == "hard":
        return num2 == hard_map.get(num1)
    
# TLX string value to int mapping
def TLX(value, measure_type):
    
    M_map = {
        "Zelo nizko": 1,
        "Nizko": 2,
        "Srednje": 3,
        "Visoko": 4,
        "Zelo visoko": 5,
    }
    
    P_map = {
        "Zelo dobro": 1,
        "Dobro": 2,
        "Srednje": 3,
        "Slabo": 4,
        "Zelo slabo": 5,
    }
    
    E_map = {
        "Zelo malo": 1,
        "Malo": 2,
        "Srednje": 3,
        "Veliko": 4,
        "Zelo veliko": 5,
    }
    
    if measure_type == "Physical" or measure_type == "Mental" or measure_type == "Temporal" or measure_type == "Frustration":
        return M_map.get(value)
    if measure_type == "Performance":
        return P_map.get(value)
    if measure_type == "Effort":
        return E_map.get(value)


def convert_to_epoch(value):
    datetime_object = datetime.strptime(value,'%Y:%m:%d:%H:%M:%S:%f')
    return int(datetime_object.strftime('%s'))*1000 + int(datetime_object.microsecond/1000)
    
# This function is called when a primary task end is detected
def process_end():
    global state_prim
    global start_time
    global end_time
    global task_type
    global label
    global finished
    global num_all_correct
    global num_correct
    global num_incorrect
    global curr_task_id
    global client_id
    global task_load_index
    global old_task_load_index
    global cursor
    
    time_on_task = end_time - start_time

    if task_type == 'HP':
        num_all_correct = 21
    if task_type == 'FA':
        #We know that there are 2 groups, 5 A-words in evey group
        num_all_correct = 10
    if task_type == 'GC':
        num_all_correct = 5
    if task_type == 'NC':
        if label == "low":
            num_all_correct = 11
        elif label == "medium":
            num_all_correct = 14
        elif label == "high":
            num_all_correct = 4
    if task_type == 'SX':
        num_all_correct = 20
    if task_type == 'PT':
        num_all_correct = 10  
    
    if curr_task_id > -1:
        print "Old: "+str(old_task_load_index)+" new: "+str(task_load_index) 
        print curr_task_id, client_id, task_type, label, start_time, time_on_task, num_correct, num_incorrect, num_all_correct, task_load_index, mental, physical, temporal, performance, effort, frustration, finished
        if cursor != None:
            cursor.execute("""UPDATE primary_task SET task_load_index = %s, mental = %s, physical = %s, temporal = %s, performance = %s, effort = %s, frustration = %s WHERE task_id = %s AND client_id = %s AND time_start = %s""",
                          (task_load_index,
                          mental, physical, temporal, performance, effort, frustration,
                          curr_task_id, client_id,start_time))
        #if cursor != None:
        #    cursor.execute("""INSERT INTO primary_task (task_id, client_id, type, label, time_on_task, time_start, time_end, num_correct, num_incorrect, num_all_correct, task_load_index, mental, physical, temporal, performance, effort, frustration, finished) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", 
        #        (curr_task_id,
        #        client_id,
        #        task_type,
        #        label,
        #        time_on_task,
        #        start_time,
        #        end_time,
        #        num_correct,
        #        num_incorrect,
        #        num_all_correct,
        #        task_load_index,
        #        mental, 
        #        physical,
        #        temporal, 
        #        performance,
        #        effort,
        #        frustration,
        #        finished))


In [37]:
connect_to_db()


Connecting to database
  ->host=127.0.0.1 port=5432 dbname=veljkodb user=veljkopg password=pntdcn15
Connected!



<cursor object at 0x7f4e8e0eee50; closed: 0>

In [38]:
# This is the main cell:
# - refer it to a directory where secondary task logs are stored 
# - it iterates through each log, recognises tasks and puts them in the table
# - DON'T forget to run the following cell in order to confirm the changes to the database.

global client_id

reset_global()
reset_task()

connect_to_db()

directory = "/home/veljko/cogload/data/sec_task" # "/home/veljko/cogload/data/ijs/logs"
mSF = re.compile(r"(\w+)-secondary.txt" )

for filename in os.listdir(directory):
    if mSF.search(filename): 
        reset_user()
        reset_task()
        client_id = mSF.search(filename).group(1)
        f = open(os.path.join(directory, filename),"r")
        for line in f:
            #print line
            parse(line)
        continue
    else:
        continue

Connecting to database
  ->host=127.0.0.1 port=5432 dbname=veljkodb user=veljkopg password=pntdcn15
Connected!

Secondary task 1 1 hpbxa 1505460296000 False
Secondary task 2 1 hpbxa 1505460417000 0.392156862745 True
Secondary task 3 1 hpbxa 1505460440000 0.156862745098 True
Secondary task 4 1 hpbxa 1505460457000 0.0941176470588 True
Secondary task 5 1 hpbxa 1505460463000 0.243137254902 True
Secondary task 6 1 hpbxa 1505460475000 0.149019607843 True
Old: 13 new: 13
1 hpbxa HP high 1505460414123 73438 19 0 21 13 3 2 2 2 3 1 True
Secondary task 7 2 hpbxa 1505460559000 0.156862745098 True
Secondary task 8 2 hpbxa 1505460573000 0.117647058824 True
Secondary task 9 2 hpbxa 1505460579000 0.078431372549 True
Secondary task 10 2 hpbxa 1505460589000 0.0470588235294 True
Old: 13 new: 11
2 hpbxa HP low 1505460551521 46727 21 0 21 11 3 1 2 2 2 1 True
Secondary task 11 3 hpbxa 1505460660000 0.109803921569 True
Secondary task 12 3 hpbxa 1505460669000 0.0705882352941 True
Secondary task 13 3 hpbxa 150

Secondary task 136 23 7swyk 1503321561000 0.141176470588 True
Secondary task 137 23 7swyk 1503321573000 0.149019607843 True
Secondary task 138 23 7swyk 1503321586000 0.133333333333 True
Secondary task 139 23 7swyk 1503321600000 0.250980392157 True
Secondary task 140 23 7swyk 1503321617000 0.0862745098039 True
Secondary task 141 23 7swyk 1503321627000 0.117647058824 True
Secondary task 142 23 7swyk 1503321634000 0.219607843137 True
Secondary task 143 23 7swyk 1503321646000 0.235294117647 True
Old: 16 new: 16
23 7swyk FA medium 1503321556400 112330 9 0 10 16 2 3 2 3 3 3 True
Secondary task 145 24 7swyk 1503321729000 0.243137254902 True
Secondary task 146 24 7swyk 1503321741000 0.0941176470588 True
Secondary task 147 24 7swyk 1503321753000 0.149019607843 True
Secondary task 148 24 7swyk 1503321763000 0.117647058824 True
Secondary task 149 24 7swyk 1503321773000 0.0862745098039 True
Secondary task 150 24 7swyk 1503321784000 0.101960784314 True
Secondary task 151 24 7swyk 1503321793000 0.07

Secondary task 283 44 1mpau 1505833957000 0.078431372549 True
Secondary task 284 44 1mpau 1505833963000 0.0392156862745 True
Secondary task 285 44 1mpau 1505833968000 0.0627450980392 True
Old: 17 new: 17
44 1mpau GC high 1505833880674 91794 0 0 5 17 3 2 3 4 3 2 True
Secondary task 286 45 1mpau 1505834015000 0.0627450980392 True
Secondary task 287 45 1mpau 1505834025000 0.0862745098039 True
Secondary task 288 45 1mpau 1505834032000 0.0392156862745 True
Secondary task 289 45 1mpau 1505834035000 0.0549019607843 True
Old: 11 new: 9
45 1mpau GC low 1505834013546 25743 0 0 5 9 1 1 2 2 2 1 True
Secondary task 290 46 1mpau 1505834107000 0.0705882352941 True
Secondary task 291 46 1mpau 1505834117000 0.0470588235294 True
Secondary task 292 46 1mpau 1505834123000 0.0627450980392 True
Secondary task 293 46 1mpau 1505834135000 0.0392156862745 True
Old: 13 new: 11
46 1mpau NC low 1505834097258 44993 12 1 11 11 2 1 2 2 3 1 True
Secondary task 295 47 1mpau 1505834193000 0.0705882352941 True
Secondary 

Secondary task 413 61 2gu87 1503929899000 0.078431372549 True
Secondary task 414 61 2gu87 1503929904000 0.0627450980392 True
Secondary task 415 61 2gu87 1503929914000 0.0470588235294 True
Secondary task 416 61 2gu87 1503929918000 0.0392156862745 True
Secondary task 417 61 2gu87 1503929922000 0.0313725490196 True
Secondary task 418 61 2gu87 1503929927000 0.0392156862745 True
Secondary task 419 61 2gu87 1503929931000 0.0470588235294 True
Secondary task 420 61 2gu87 1503929937000 0.0627450980392 True
Secondary task 421 61 2gu87 1503929947000 0.0941176470588 True
Secondary task 422 61 2gu87 1503929960000 0.0705882352941 True
Secondary task 423 61 2gu87 1503929969000 0.0627450980392 True
Secondary task 424 61 2gu87 1503929979000 0.0470588235294 True
Secondary task 425 61 2gu87 1503929986000 0.078431372549 True
Secondary task 426 61 2gu87 1503929990000 0.0549019607843 True
Secondary task 427 61 2gu87 1503930002000 0.117647058824 True
Old: 24 new: 17
61 2gu87 GC medium 1503929866255 146553 0 

Secondary task 566 81 tn4vl 1505486844000 0.078431372549 True
Secondary task 567 81 tn4vl 1505486857000 0.0627450980392 True
Secondary task 568 81 tn4vl 1505486863000 0.125490196078 True
Old: 12 new: 12
81 tn4vl HP high 1505486796874 74375 20 1 21 12 3 1 2 1 3 2 True
Secondary task 569 82 tn4vl 1505486919000 0.0862745098039 True
Secondary task 570 82 tn4vl 1505486933000 0.078431372549 True
Secondary task 571 82 tn4vl 1505486939000 0.211764705882 True
Secondary task 572 82 tn4vl 1505486954000 0.0941176470588 True
Secondary task 573 82 tn4vl 1505486968000 0.0549019607843 True
Old: 10 new: 10
82 tn4vl HP low 1505486916276 56080 21 0 21 10 2 1 2 1 2 2 True
Secondary task 574 83 tn4vl 1505487023000 0.0392156862745 True
Secondary task 575 83 tn4vl 1505487030000 0.117647058824 True
Secondary task 576 83 tn4vl 1505487042000 0.125490196078 True
Secondary task 577 83 tn4vl 1505487054000 0.0705882352941 True
Secondary task 578 83 tn4vl 1505487064000 0.117647058824 True
Secondary task 579 83 tn4vl

...

Secondary task 1107 152 iz2ps 1503318312000 0.0941176470588 True
Secondary task 1108 152 iz2ps 1503318325000 0.0627450980392 True
Secondary task 1109 152 iz2ps 1503318331000 0.0862745098039 True
Old: 21 new: 21
152 iz2ps PT high 1503318254500 86185 0 10 10 21 4 4 3 3 4 3 True
Secondary task 1117 153 c24ur 1505318662000 False
Secondary task 1118 153 c24ur 1505318818000 0.172549019608 True
Secondary task 1119 153 c24ur 1505318828000 0.188235294118 True
Secondary task 1120 153 c24ur 1505318838000 0.0549019607843 True
Secondary task 1121 153 c24ur 1505318848000 0.0470588235294 True
Secondary task 1122 153 c24ur 1505318855000 0.101960784314 True
Secondary task 1123 153 c24ur 1505318863000 0.0470588235294 True
Secondary task 1124 153 c24ur 1505318870000 0.0627450980392 True
Secondary task 1125 153 c24ur 1505318881000 0.141176470588 True
Secondary task 1126 153 c24ur 1505318892000 0.101960784314 True
Secondary task 1127 153 c24ur 1505318905000 0.0470588235294 True
Secondary task 1128 153 c24u

Secondary task 1246 164 c24ur 1505320648000 0.0470588235294 True
Secondary task 1247 164 c24ur 1505320656000 0.0549019607843 True
Secondary task 1248 164 c24ur 1505320665000 0.0941176470588 True
Secondary task 1249 164 c24ur 1505320670000 0.0392156862745 True
Secondary task 1250 164 c24ur 1505320675000 0.0627450980392 True
Secondary task 1251 164 c24ur 1505320681000 0.149019607843 True
Secondary task 1252 164 c24ur 1505320691000 0.0313725490196 True
Secondary task 1253 164 c24ur 1505320697000 0.0705882352941 True
Secondary task 1254 164 c24ur 1505320708000 0.0549019607843 True
Secondary task 1255 164 c24ur 1505320714000 0.0549019607843 True
Secondary task 1256 164 c24ur 1505320723000 0.0470588235294 True
Secondary task 1257 164 c24ur 1505320736000 0.0549019607843 True
Old: 32 new: 9
164 c24ur NC medium 1505320642107 102484 14 0 14 9 1 1 1 3 2 1 True
Secondary task 1259 165 c24ur 1505320912000 0.211764705882 True
Secondary task 1260 165 c24ur 1505320924000 0.0392156862745 True
Secondary

Secondary task 1390 181 e4gay 1503328245000 0.109803921569 True
Secondary task 1391 181 e4gay 1503328252000 0.0549019607843 True
Secondary task 1392 181 e4gay 1503328258000 0.0862745098039 True
Secondary task 1393 181 e4gay 1503328271000 0.0392156862745 True
Secondary task 1394 181 e4gay 1503328283000 0.0470588235294 True
Secondary task 1395 181 e4gay 1503328293000 0.0470588235294 True
Secondary task 1396 181 e4gay 1503328298000 0.0470588235294 True
Secondary task 1397 181 e4gay 1503328301000 0.0470588235294 True
Secondary task 1398 181 e4gay 1503328306000 0.0470588235294 True
Secondary task 1399 181 e4gay 1503328311000 0.0549019607843 True
Secondary task 1400 181 e4gay 1503328316000 0.0392156862745 True
Secondary task 1401 181 e4gay 1503328324000 0.0470588235294 True
Secondary task 1402 181 e4gay 1503328334000 0.0862745098039 True
Secondary task 1403 181 e4gay 1503328343000 0.117647058824 True
Old: 28 new: 18
181 e4gay NC high 1503328201897 159326 4 1 4 18 4 2 4 2 4 2 True
Secondary t

Secondary task 1547 202 ef5rq 1504020422000 0.188235294118 True
Secondary task 1548 202 ef5rq 1504020433000 0.219607843137 True
Secondary task 1549 202 ef5rq 1504020443000 0.125490196078 True
Secondary task 1550 202 ef5rq 1504020455000 0.141176470588 True
Secondary task 1551 202 ef5rq 1504020469000 0.0862745098039 True
Old: 13 new: 12
202 ef5rq SX medium 1504020415753 57946 20 0 20 12 3 3 1 2 2 1 True
Secondary task 1552 203 ef5rq 1504020522000 0.407843137255 True
Secondary task 1553 203 ef5rq 1504020540000 0.164705882353 True
Secondary task 1554 203 ef5rq 1504020553000 0.0705882352941 True
Secondary task 1555 203 ef5rq 1504020566000 0.0470588235294 True
Secondary task 1556 203 ef5rq 1504020578000 0.0313725490196 True
Secondary task 1557 203 ef5rq 1504020582000 0.0392156862745 True
Secondary task 1558 203 ef5rq 1504020592000 0.0549019607843 True
Secondary task 1559 203 ef5rq 1504020596000 0.0392156862745 True
Old: 26 new: 14
203 ef5rq SX high 1504020517233 81551 20 0 20 14 2 4 2 2 3 1 

Secondary task 1705 221 f1gjp 1503934471000 0.117647058824 True
Old: 19 new: 19
221 f1gjp SX high 1503934342376 134831 20 0 20 19 3 3 3 3 4 3 True
Secondary task 1706 222 f1gjp 1503934596000 0.376470588235 True
Secondary task 1707 222 f1gjp 1503934615000 0.0862745098039 True
Secondary task 1708 222 f1gjp 1503934621000 0.0941176470588 True
Secondary task 1709 222 f1gjp 1503934635000 0.0862745098039 True
Old: 11 new: 11
222 f1gjp PT medium 1503934588520 56600 9 1 10 11 2 2 1 1 2 3 True
Secondary task 1711 223 f1gjp 1503934720000 0.0627450980392 True
Secondary task 1712 223 f1gjp 1503934733000 0.101960784314 True
Secondary task 1713 223 f1gjp 1503934744000 0.0627450980392 True
Secondary task 1714 223 f1gjp 1503934757000 0.078431372549 True
Old: 10 new: 10
223 f1gjp PT low 1503934712520 49172 10 0 10 10 2 2 1 2 2 1 True
Secondary task 1715 224 f1gjp 1503934826000 0.078431372549 True
Secondary task 1716 224 f1gjp 1503934831000 0.109803921569 True
Secondary task 1717 224 f1gjp 1503934843000 

In [39]:
# Run this to commit the changes 
# Alternatively, uncomment "rollback" and comment "commit" in order to start again.

conn.commit()
#conn.rollback()
cursor.close()
conn.close()